## 1. Data Loading 

In [ ]:
# =====================================
# 📘 Thesis Project - Dropout Prediction
# Section: 1. Data Loading & Preparation
# Author: Tobias Benavides
# Date: March 21, 2025
# =====================================

# 🔧 Imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostClassifier
import os

print("🔥 Everything is working in the right environment!")

# 🧾 Project Context
"""
This notebook is part of the thesis project: 
"Where did the students go? Leveraging Oversampling and Neural Networks to Predict University Dropouts".

Objective:
- Load and explore the dataset from the UCI Machine Learning Repository.
- Prepare the data for further preprocessing and modeling stages.
- Ensure modularity and flexibility in case the dataset is updated or replaced.
"""

# 📂 Dataset Path (change this if needed)
DATA_PATH = "data/data.csv"

# ✅ Load Dataset
try:
    df = pd.read_csv(DATA_PATH, delimiter=';')
    print(f"✅ Data successfully loaded! Shape: {df.shape}")
    df_cleaned = df.copy()  # Save a clean working copy
except FileNotFoundError:
    print(f"❌ File not found at path: {DATA_PATH}")
    df = None

# ✅ Initial Cleaning & Checks
if df is not None:
    df.columns = df.columns.str.strip()  # clean column names
    display(df.head())
    display(df.info())
    display(df.describe(include='all'))

    # Check missing values and duplicates
    print("🔍 Missing values per column:\n", df.isnull().sum())
    print(f"🔁 Duplicate rows found: {df.duplicated().sum()}")

    # Remove 'Enrolled' class and encode target
    df = df[df["Target"] != "Enrolled"]
    df["Target_Encoded"] = df["Target"].map({"Dropout": 0, "Graduate": 1})
    df.drop(columns=["Target"], inplace=True)
    df.reset_index(drop=True, inplace=True)

    # Split features and target
    X = df.drop(columns=["Target_Encoded"])
    y = df["Target_Encoded"]

    # Visual sanity check
    sns.countplot(x=y)
    plt.title("Distribution of Dropout vs Graduate")
    plt.xlabel("Target Class")
    plt.ylabel("Count")
    plt.xticks([0, 1], ['Dropout', 'Graduate'])
    plt.show()


### 2) First EDA


In [ ]:
# =====================================
# 📊 Section: Exploratory Data Analysis (EDA)
# Including: Target Encoding + Feature Grouping
# =====================================

# 🎯 Confirm target column and encode
TARGET_COL = 'Target'
label_mapping = {'Dropout': 0, 'Graduate': 1, 'Enrolled': 2}  # Enrolled will be removed later
df_cleaned['Target_Encoded'] = df_cleaned[TARGET_COL].map(label_mapping)

# 🟢 Class Distribution Plot (original labels)
target_counts = df_cleaned[TARGET_COL].value_counts(normalize=True).sort_index()
plt.figure(figsize=(8, 5))
sns.barplot(x=target_counts.index, y=target_counts.values)
plt.title("🎯 Target Class Distribution")
plt.ylabel("Proportion")
plt.xlabel("Class Label")
plt.show()

print("\n📈 Class Distribution:")
display(df_cleaned[TARGET_COL].value_counts())

# 🗂️ Feature Groups (based on thesis proposal)
demographic_cols = ['Gender', 'Age at enrollment', 'Nationality', 'Marital status']
socioeconomic_cols = ["Scholarship holder", "Father's occupation", "Mother's qualification"]
academic_cols = [
    'Curricular units 1st sem (enrolled)', 'Curricular units 1st sem (approved)', 
    'Curricular units 2nd sem (enrolled)', 'Curricular units 2nd sem (approved)'
]
macroeconomic_cols = ['Unemployment rate', 'Inflation rate', 'GDP']

# 🎨 Plotting Helper Function
def plot_feature_distributions(feature_list, title_prefix):
    for col in feature_list:
        if col in df_cleaned.columns:
            plt.figure(figsize=(7, 4))
            if df_cleaned[col].dtype in ['object', 'category']:
                sns.countplot(data=df_cleaned, x=col, hue=TARGET_COL)
            else:
                sns.boxplot(data=df_cleaned, x=TARGET_COL, y=col)
            plt.title(f"{title_prefix}: {col}")
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()

# 🧬 Demographic Features
plot_feature_distributions(demographic_cols, "🧬 Demographic")

# 💰 Socio-Economic Features
plot_feature_distributions(socioeconomic_cols, "💰 Socio-Economic")

# 🎓 Academic Features
plot_feature_distributions(academic_cols, "🎓 Academic")

# 🌍 Macroeconomic Features
plot_feature_distributions(macroeconomic_cols, "🌍 Macroeconomic")

# 🔍 Correlation Matrix (numeric only, including encoded target)
numerical_df = df_cleaned.select_dtypes(include='number')

if numerical_df.shape[1] == 0:
    print("⚠️ No numerical columns found — skipping correlation matrix.")
else:
    plt.figure(figsize=(12, 10))
    correlation = numerical_df.corr()
    sns.heatmap(correlation, annot=False, cmap='coolwarm', fmt=".2f", center=0)
    plt.title("🔍 Correlation Matrix of Numerical Features")
    plt.show()

### 3) Further feature EDA

In [ ]:
# =====================================
# 🔍 Enhanced EDA: Advanced Feature Insights
# =====================================

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from scipy.stats import kruskal, chi2_contingency, shapiro
from statsmodels.stats.outliers_influence import variance_inflation_factor
import missingno as msno
import warnings
warnings.filterwarnings('ignore')

# ✅ Setup
df_eda = df_cleaned.copy()  # df_cleaned already has 'Target_Encoded'

# 🎯 Drop 'Target' for clarity if it exists
if "Target" in df_eda.columns:
    df_eda.drop(columns=["Target"], inplace=True)

# 1. 🔎 Missing Value Matrix
msno.matrix(df_eda)
plt.title("Missing Values Matrix")
plt.show()

# 2. 🧠 Multicollinearity - Variance Inflation Factor (VIF)
numeric_features = df_eda.select_dtypes(include=np.number).drop(columns=['Target_Encoded'])
scaler = StandardScaler()
X_scaled = scaler.fit_transform(numeric_features)
vif_data = pd.DataFrame({
    'Feature': numeric_features.columns,
    'VIF': [variance_inflation_factor(X_scaled, i) for i in range(X_scaled.shape[1])]
}).sort_values(by="VIF", ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(data=vif_data, x='VIF', y='Feature')
plt.title("Variance Inflation Factor (VIF)")
plt.tight_layout()
plt.show()

# 3. 📊 Feature Importance via Random Forest
X = df_eda.drop(columns=['Target_Encoded'])
y = df_eda['Target_Encoded']
rf = RandomForestClassifier(random_state=42)
rf.fit(X, y)
importances = pd.DataFrame({
    'Feature': X.columns,
    'Importance': rf.feature_importances_
}).sort_values(by="Importance", ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(data=importances.head(15), x='Importance', y='Feature')
plt.title("Top 15 Feature Importances (Random Forest)")
plt.tight_layout()
plt.show()

# 4. 📈 Kruskal-Wallis Test (for numeric vs Target_Encoded)
print("🔬 Kruskal-Wallis Test (Numerical features vs Target):")
for col in numeric_features.columns:
    groups = [df_eda[df_eda["Target_Encoded"] == val][col] for val in df_eda["Target_Encoded"].unique()]
    stat, p = kruskal(*groups)
    print(f" - {col}: H = {stat:.2f}, p = {p:.4f}")

# 5. 🧪 Chi-Square Test (for categorical vs Target_Encoded)
print("\n🔬 Chi-Square Test (Categorical features vs Target):")
categorical_cols = df_eda.select_dtypes(include='int64').columns.difference(numeric_features.columns)
for col in categorical_cols:
    contingency = pd.crosstab(df_eda[col], df_eda["Target_Encoded"])
    chi2, p, _, _ = chi2_contingency(contingency)
    print(f" - {col}: Chi2 = {chi2:.2f}, p = {p:.4f}")

# 6. 🧬 Normality & Skewness Check
print("\n📏 Skewness & Normality Test (Shapiro-Wilk):")
for col in numeric_features.columns:
    skew_val = df_eda[col].skew()
    _, p_shapiro = shapiro(df_eda[col])
    print(f" - {col}: skew = {skew_val:.2f}, Shapiro p = {p_shapiro:.4f}")

# 7. 🎯 PCA Visualization
pca = PCA(n_components=2)
pca_result = pca.fit_transform(scaler.fit_transform(numeric_features))
pca_df = pd.DataFrame(pca_result, columns=['PC1', 'PC2'])
pca_df['Target'] = df_eda['Target_Encoded']

plt.figure(figsize=(8, 6))
sns.scatterplot(data=pca_df, x='PC1', y='PC2', hue='Target', palette='coolwarm')
plt.title("PCA Projection of Numerical Features")
plt.tight_layout()
plt.show()

# 8. 🧊 Outlier Detection via Boxplots
plt.figure(figsize=(14, 6))
sns.boxplot(data=numeric_features)
plt.xticks(rotation=90)
plt.title("Boxplots for Outlier Detection (Numeric Features)")
plt.tight_layout()
plt.show()

# 9. 🔗 Pairplot (Feature Relationships + Target)
selected_pairplot_cols = numeric_features.columns[:4].tolist() + ['Target_Encoded']
sns.pairplot(df_eda[selected_pairplot_cols], hue='Target_Encoded')
plt.suptitle("Pairwise Feature Relationships", y=1.02)
plt.show()


### 3.1 Dropping the "Enrolled" class

In [ ]:
# 🛠️ Section: Feature Engineering & Preprocessing (Improved)
# =====================================

from sklearn.preprocessing import LabelEncoder

# 1. ✅ Remove "Enrolled" class from dataset (for final modeling)
df_model = df_cleaned[df_cleaned['Target'] != 'Enrolled'].copy()
print(f"✅ Shape after removing 'Enrolled': {df_model.shape}")

# 2. 🎯 Encode target variable: Dropout = 0, Graduate = 1
target_map = {'Dropout': 0, 'Graduate': 1}
df_model['Target_Encoded'] = df_model['Target'].map(target_map)

# (Optional) Remove original target column now to avoid drop errors later
df_model.drop(columns='Target', inplace=True)

# 3. 🧼 Encode categorical variables using LabelEncoder
# Safe column selection with error-tolerant drop
categorical_cols = df_model.select_dtypes(include=['object']).columns
le_dict = {}

for col in categorical_cols:
    le = LabelEncoder()
    df_model[col] = le.fit_transform(df_model[col])
    le_dict[col] = le  # Save encoders for inverse transform later

print(f"✅ Categorical encoding complete. Encoded columns: {list(categorical_cols)}")

# 4. 🧪 Split features/target
X = df_model.drop(columns='Target_Encoded')
y = df_model['Target_Encoded']

# Save feature names for reference
feature_names = X.columns.tolist()

print(f"📊 Feature matrix shape: {X.shape}")
print(f"🎯 Target vector shape: {y.shape}")
print("🎯 Target class distribution:\n", y.value_counts())

### 3.2 New Class Distribution Graph  

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 🟦 Plot target class distribution
plt.figure(figsize=(6, 4))
sns.countplot(x=y, palette='pastel')
plt.title("🎯 Final Target Class Distribution")
plt.xlabel("Target Class (0 = Dropout, 1 = Graduate)")
plt.ylabel("Number of Students")
plt.xticks(ticks=[0, 1], labels=['Dropout', 'Graduate'])
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


### 3.3) Enhanced EDA (Post-dropping "Enrolled" class)


In [ ]:
# =====================================
# 🔍 Enhanced EDA: Post-Preprocessing Analysis
# =====================================

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.stats import kruskal, chi2_contingency, shapiro
from statsmodels.stats.outliers_influence import variance_inflation_factor
import missingno as msno
import warnings
warnings.filterwarnings('ignore')

# 1. 🧼 Missing Value Matrix
msno.matrix(df_model)
plt.title("Missing Values Matrix")
plt.show()

# 2. 🧠 Variance Inflation Factor (VIF)
X_scaled = StandardScaler().fit_transform(X)
vif_data = pd.DataFrame({
    'Feature': feature_names,
    'VIF': [variance_inflation_factor(X_scaled, i) for i in range(X_scaled.shape[1])]
}).sort_values(by="VIF", ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(data=vif_data, x='VIF', y='Feature')
plt.title("Multicollinearity Check: Variance Inflation Factor (VIF)")
plt.tight_layout()
plt.show()

# 3. 🌲 Feature Importance via Random Forest
rf = RandomForestClassifier(random_state=42)
rf.fit(X, y)
importances = pd.DataFrame({
    'Feature': feature_names,
    'Importance': rf.feature_importances_
}).sort_values(by="Importance", ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(data=importances.head(15), x='Importance', y='Feature')
plt.title("Top 15 Feature Importances (Random Forest)")
plt.tight_layout()
plt.show()

# 4. 🧪 Kruskal-Wallis Test (Numerical vs Target)
print("🔬 Kruskal-Wallis Test (Numerical features vs Target):")
numeric_cols = X.select_dtypes(include=np.number).columns
for col in numeric_cols:
    groups = [X[y == val][col] for val in sorted(y.unique())]
    stat, p = kruskal(*groups)
    print(f" - {col}: H = {stat:.2f}, p = {p:.4f}")

# 5. 🧪 Chi-Square Test (Categorical vs Target)
print("\n🔬 Chi-Square Test (Categorical features vs Target):")
categorical_cols = X.select_dtypes(include='int64').columns.difference(numeric_cols)
for col in categorical_cols:
    contingency = pd.crosstab(X[col], y)
    chi2, p, _, _ = chi2_contingency(contingency)
    print(f" - {col}: Chi2 = {chi2:.2f}, p = {p:.4f}")

# 6. 🧪 Skewness & Normality Test (Shapiro-Wilk)
print("\n📏 Skewness & Normality:")
for col in numeric_cols:
    skew_val = X[col].skew()
    _, p_shapiro = shapiro(X[col])
    print(f" - {col}: skew = {skew_val:.2f}, Shapiro p = {p_shapiro:.4f}")

# 7. 🌐 PCA Projection
pca = PCA(n_components=2)
pca_result = pca.fit_transform(X_scaled)
pca_df = pd.DataFrame(pca_result, columns=['PC1', 'PC2'])
pca_df['Target'] = y.values

plt.figure(figsize=(8, 6))
sns.scatterplot(data=pca_df, x='PC1', y='PC2', hue='Target', palette='coolwarm')
plt.title("PCA Projection of Feature Space")
plt.tight_layout()
plt.show()

# 8. 🧊 Outlier Detection via Boxplots
plt.figure(figsize=(14, 6))
sns.boxplot(data=X[numeric_cols])
plt.xticks(rotation=90)
plt.title("Outlier Detection (Boxplots of Numeric Features)")
plt.tight_layout()
plt.show()

# 9. 🔗 Pairplot of Top 4 Numeric Features
top4 = importances.head(4)['Feature'].tolist()
sns.pairplot(df_model[top4 + ['Target_Encoded']], hue='Target_Encoded')
plt.suptitle("Pairwise Relationships of Top Features", y=1.02)
plt.show()


### 4) Actual Feature engeeniring

In [ ]:
# =============================================
# 🧠 Final Feature Engineering + PCA + SHAP (Safe Version)
# =============================================

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
import shap
import matplotlib.pyplot as plt
import seaborn as sns

# Step 1: Filter and encode target
df_model = df_cleaned[df_cleaned['Target'] != 'Enrolled'].copy()
df_model['Target_Encoded'] = df_model['Target'].map({'Dropout': 0, 'Graduate': 1})
df_model.drop(columns='Target', inplace=True)

# Step 2: Drop irrelevant/redundant features
drop_cols = [
    "Course", "Nacionality", "International", "Educational special needs",
    "Curricular units 1st sem (credited)", "Curricular units 2nd sem (credited)",
    "Mother's occupation", "Father's occupation", "Application order",
    "Mother's qualification", "Father's qualification"
]
df_model.drop(columns=[col for col in drop_cols if col in df_model.columns], inplace=True)

# Step 3: Label encode categorical features
le_dict = {}
categorical_cols = df_model.select_dtypes(include='object').columns
for col in categorical_cols:
    le = LabelEncoder()
    df_model[col] = le.fit_transform(df_model[col])
    le_dict[col] = le

# Step 4: Composite academic performance features
df_model['performance_1st'] = df_model['Curricular units 1st sem (approved)'] + df_model['Curricular units 1st sem (grade)']
df_model['performance_2nd'] = df_model['Curricular units 2nd sem (approved)'] + df_model['Curricular units 2nd sem (grade)']
df_model.drop(columns=[
    'Curricular units 1st sem (approved)', 'Curricular units 1st sem (grade)',
    'Curricular units 2nd sem (approved)', 'Curricular units 2nd sem (grade)'
], inplace=True)

# Step 5: Bin skewed variables
df_model['age_bin'] = pd.cut(df_model['Age at enrollment'], bins=[15, 20, 25, 30, 50], labels=[0, 1, 2, 3])
df_model['admission_bin'] = pd.cut(df_model['Admission grade'], bins=[0, 10, 13, 20], labels=[0, 1, 2])
df_model.drop(columns=['Age at enrollment', 'Admission grade'], inplace=True)

# Step 6: Select final feature set
selected_features = [
    'performance_1st', 'performance_2nd',
    'Curricular units 1st sem (evaluations)', 'Curricular units 2nd sem (evaluations)',
    'Tuition fees up to date', 'Debtor', 'Scholarship holder',
    'Gender', 'age_bin', 'admission_bin'
]
X = df_model[selected_features].copy()
y = df_model['Target_Encoded']
feature_names = X.columns.tolist()

print(f"✅ Final shape of X: {X.shape}")
print(f"✅ Final shape of y: {y.shape}")
print(f"📋 Features used: {feature_names}")

# Step 7: PCA for 2D visualization
X_encoded = pd.get_dummies(X, drop_first=True).astype(float)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_encoded)

pca = PCA(n_components=2)
pca_components = pca.fit_transform(X_scaled)
pca_df = pd.DataFrame(pca_components, columns=['PC1', 'PC2'])
pca_df['Target'] = y.values

plt.figure(figsize=(8, 6))
sns.scatterplot(data=pca_df, x='PC1', y='PC2', hue='Target', palette='coolwarm')
plt.title("🎯 PCA Projection of Final Feature Space")
plt.tight_layout()
plt.show()

# Step 8: SHAP Explanations (Robust fix for binary classification)
model = RandomForestClassifier(random_state=42)
model.fit(X_encoded, y)

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_encoded)

# Handle SHAP values structure (list for binary classification)
if isinstance(shap_values, list):
    shap_vals = shap_values[1]  # class = Graduate
else:
    shap_vals = shap_values

# Check shapes before plotting
print("🔍 SHAP matrix shape:", np.array(shap_vals).shape)
print("📊 Feature matrix shape:", X_encoded.shape)

# SHAP summary plot
shap.summary_plot(shap_vals, features=X_encoded, feature_names=X_encoded.columns)


# 5) Baseline model comparison

### 5.1) Random Forrest - Baseline

In [ ]:
# ============================================
# 🌲 Random Forest Baseline + Full Evaluation
# Using Final Feature Selection Set
# ============================================

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import (
    classification_report, confusion_matrix, ConfusionMatrixDisplay,
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, RocCurveDisplay,
    mean_squared_error, log_loss
)
import matplotlib.pyplot as plt
import numpy as np

# 1. ✂️ Train-test split (80/20) — stratify to preserve class distribution
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, stratify=y, random_state=42
)

# 2. 🌲 Initialize Random Forest
rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=1000,
    class_weight='balanced',
    random_state=42
)

# 3. 🧠 Train model
rf.fit(X_train, y_train)

# 4. 🔍 Predict
y_pred = rf.predict(X_test)
y_proba = rf.predict_proba(X_test)[:, 1]

# 5. 📊 Evaluation Metrics
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)
mse = mean_squared_error(y_test, y_proba)
logloss = log_loss(y_test, y_proba)

print("📊 Evaluation on Test Set:")
print(f"✅ Accuracy:       {acc:.4f}")
print(f"✅ Precision:      {prec:.4f}")
print(f"✅ Recall:         {rec:.4f}")
print(f"✅ F1 Score:       {f1:.4f}")
print(f"✅ ROC-AUC:        {roc_auc:.4f}")
print(f"✅ MSE:            {mse:.4f}")
print(f"✅ Log Loss:       {logloss:.4f}")

# 6. 📜 Classification Report
print("\n📋 Classification Report:")
print(classification_report(y_test, y_pred, target_names=["Dropout", "Graduate"]))

# 7. 🔁 Confusion Matrix
ConfusionMatrixDisplay.from_estimator(
    rf, X_test, y_test, display_labels=["Dropout", "Graduate"], cmap='Blues'
)
plt.title("🔍 Confusion Matrix: Random Forest (Final Feature Set)")
plt.grid(False)
plt.show()

# 8. 📈 ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_proba)
plt.figure()
RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name="Random Forest").plot()
plt.title("📈 ROC Curve: Random Forest (Final Feature Set)")
plt.grid(True)
plt.show()

# 9. 🔁 Cross-validated scores (5-fold)
cv_f1 = cross_val_score(rf, X_encoded, y, scoring='f1', cv=5)
cv_auc = cross_val_score(rf, X_encoded, y, scoring='roc_auc', cv=5)

print(f"\n🔁 Cross-validated F1 Scores:       {cv_f1}")
print(f"📌 Mean F1 Score (5-fold):          {cv_f1.mean():.4f}")
print(f"\n🔁 Cross-validated ROC-AUC Scores:  {cv_auc}")
print(f"📌 Mean ROC-AUC Score (5-fold):     {cv_auc.mean():.4f}")


### 5.2) CatBoost - Baseline

In [ ]:
# =============================================
# 🐱 CatBoost Baseline Model — Final Feature Set
# Full Metrics + CV + ROC + Confusion Matrix
# =============================================

from catboost import CatBoostClassifier
from sklearn.metrics import (
    classification_report, confusion_matrix, ConfusionMatrixDisplay,
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, RocCurveDisplay,
    mean_squared_error, log_loss
)
from sklearn.model_selection import train_test_split, cross_val_score
import matplotlib.pyplot as plt
import numpy as np

# 1. ✂️ Train-test split (80/20) using final encoded features
X_train_cb, X_test_cb, y_train_cb, y_test_cb = train_test_split(
    X_encoded, y, test_size=0.2, stratify=y, random_state=42
)

# 2. 🐱 Initialize CatBoostClassifier
catboost_model = CatBoostClassifier(
    verbose=0,              # Suppress training output
    random_seed=42,
    eval_metric='F1',
    class_weights=[1, 1]    # Equal weights for baseline
)

# 3. 🧪 Train the model
catboost_model.fit(X_train_cb, y_train_cb)

# 4. 🔍 Predict on test set
y_pred_cb = catboost_model.predict(X_test_cb)
y_proba_cb = catboost_model.predict_proba(X_test_cb)[:, 1]

# 5. 📊 Evaluation Metrics
acc_cb = accuracy_score(y_test_cb, y_pred_cb)
prec_cb = precision_score(y_test_cb, y_pred_cb)
rec_cb = recall_score(y_test_cb, y_pred_cb)
f1_cb = f1_score(y_test_cb, y_pred_cb)
roc_auc_cb = roc_auc_score(y_test_cb, y_proba_cb)
mse_cb = mean_squared_error(y_test_cb, y_proba_cb)
logloss_cb = log_loss(y_test_cb, y_proba_cb)

print("📊 Evaluation on Test Set (CatBoost):")
print(f"✅ Accuracy:       {acc_cb:.4f}")
print(f"✅ Precision:      {prec_cb:.4f}")
print(f"✅ Recall:         {rec_cb:.4f}")
print(f"✅ F1 Score:       {f1_cb:.4f}")
print(f"✅ ROC-AUC:        {roc_auc_cb:.4f}")
print(f"✅ MSE:            {mse_cb:.4f}")
print(f"✅ Log Loss:       {logloss_cb:.4f}")

# 6. 📋 Classification Report
print("\n📋 Classification Report:")
print(classification_report(y_test_cb, y_pred_cb, target_names=["Dropout", "Graduate"]))

# 7. 🔁 Confusion Matrix
ConfusionMatrixDisplay.from_predictions(
    y_test_cb, y_pred_cb, display_labels=["Dropout", "Graduate"], cmap='Blues'
)
plt.title("🔍 Confusion Matrix: CatBoost (Final Feature Set)")
plt.grid(False)
plt.show()

# 8. 📈 ROC Curve
fpr_cb, tpr_cb, _ = roc_curve(y_test_cb, y_proba_cb)
plt.figure()
RocCurveDisplay(fpr=fpr_cb, tpr=tpr_cb, roc_auc=roc_auc_cb, estimator_name="CatBoost").plot()
plt.title("📈 ROC Curve: CatBoost (Final Feature Set)")
plt.grid(True)
plt.show()

# 9. 🔁 Cross-validated metrics (5-fold)
cv_f1_cb = cross_val_score(catboost_model, X_encoded, y, scoring='f1', cv=5)
cv_auc_cb = cross_val_score(catboost_model, X_encoded, y, scoring='roc_auc', cv=5)

print(f"\n🔁 Cross-validated F1 Scores:       {cv_f1_cb}")
print(f"📌 Mean F1 Score (5-fold):          {cv_f1_cb.mean():.4f}")
print(f"\n🔁 Cross-validated ROC-AUC Scores:  {cv_auc_cb}")
print(f"📌 Mean ROC-AUC Score (5-fold):     {cv_auc_cb.mean():.4f}")


### 4.3) ANN - Baseline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    f1_score, accuracy_score, roc_auc_score, roc_curve, auc,
    precision_score, recall_score, log_loss, mean_squared_error, confusion_matrix
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# === Setup ===
f1_scores, acc_scores, auc_scores = [], [], []
prec_scores, rec_scores, logloss_scores, mse_scores = [], [], [], []
conf_matrices = []
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

X_array = X_encoded.to_numpy()
y_array = y.to_numpy()

tprs, aucs = [], []
mean_fpr = np.linspace(0, 1, 100)
plt.figure(figsize=(8, 6))

# === Cross-Validation ===
for fold, (train_idx, test_idx) in enumerate(skf.split(X_array, y_array), 1):
    print(f"\n🔁 Fold {fold}/{n_splits}")
    X_train, X_test = X_array[train_idx], X_array[test_idx]
    y_train, y_test = y_array[train_idx], y_array[test_idx]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = Sequential([
        Dense(16, activation='relu', input_shape=(X_train_scaled.shape[1],)),
        Dropout(0.2),
        Dense(8, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=50, batch_size=32,
              callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)],
              verbose=0)

    y_pred_probs = model.predict(X_test_scaled).ravel()
    y_pred = (y_pred_probs >= 0.5).astype(int)

    acc_scores.append(accuracy_score(y_test, y_pred))
    f1_scores.append(f1_score(y_test, y_pred))
    prec_scores.append(precision_score(y_test, y_pred))
    rec_scores.append(recall_score(y_test, y_pred))
    auc_val = roc_auc_score(y_test, y_pred_probs)
    auc_scores.append(auc_val)
    logloss_scores.append(log_loss(y_test, y_pred_probs))
    mse_scores.append(mean_squared_error(y_test, y_pred_probs))
    conf_matrices.append(confusion_matrix(y_test, y_pred))

    fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
    tprs.append(np.interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    aucs.append(auc_val)
    plt.plot(fpr, tpr, alpha=0.3, label=f'Fold {fold} ROC (AUC = {auc_val:.2f})')

# === ROC Curve ===
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
mean_tpr = np.mean(tprs, axis=0)
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
plt.plot(mean_fpr, mean_tpr, color='b', label=f'Mean ROC (AUC = {mean_auc:.2f} ± {std_auc:.2f})', lw=2)
plt.title('📈 ROC Curve - ANN (5-Fold CV)')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.grid(True)
plt.tight_layout()
plt.show()

# === Averaged Confusion Matrix ===
avg_cm = np.mean(conf_matrices, axis=0).astype(int)
plt.figure(figsize=(6, 5))
sns.heatmap(avg_cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Dropout', 'Graduate'],
            yticklabels=['Dropout', 'Graduate'])
plt.title("🔁 Averaged Confusion Matrix (ANN - 5-Fold CV)")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.tight_layout()
plt.show()

# === SHAP KernelExplainer (fully CPU compatible) ===
X_train_sample = X_train_scaled[:100]
X_test_sample = X_test_scaled[:100]

predict_fn = lambda x: model.predict(x).ravel()
explainer = shap.KernelExplainer(predict_fn, X_train_sample)
shap_values = explainer.shap_values(X_test_sample)

shap.summary_plot(shap_values, features=X_test_sample, feature_names=X_encoded.columns)

# === Summary ===
print("\n📊 ANN Baseline Summary:")
print(f"✅ Accuracy:   {np.mean(acc_scores):.4f} ± {np.std(acc_scores):.4f}")
print(f"✅ Precision:  {np.mean(prec_scores):.4f} ± {np.std(prec_scores):.4f}")
print(f"✅ Recall:     {np.mean(rec_scores):.4f} ± {np.std(rec_scores):.4f}")
print(f"✅ F1 Score:   {np.mean(f1_scores):.4f} ± {np.std(f1_scores):.4f}")
print(f"✅ ROC-AUC:    {np.mean(auc_scores):.4f} ± {np.std(auc_scores):.4f}")
print(f"✅ MSE:        {np.mean(mse_scores):.4f} ± {np.std(mse_scores):.4f}")
print(f"✅ Log Loss:   {np.mean(logloss_scores):.4f} ± {np.std(logloss_scores):.4f}")


# 5. SMOTE Oversampling Comparisons

## 5.1 Random Forrest + SMOTE

In [ ]:
# =========================================
# 🌲 Random Forest with SMOTE — Enhanced Metrics + CV (Fixed)
# =========================================

from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    classification_report, confusion_matrix, ConfusionMatrixDisplay,
    roc_auc_score, roc_curve, RocCurveDisplay,
    accuracy_score, f1_score, precision_score, recall_score,
    mean_squared_error, log_loss
)
from sklearn.model_selection import train_test_split, cross_val_score
import matplotlib.pyplot as plt

# 1. Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# 2. Impute missing values
imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)
X_imputed = imputer.fit_transform(X)  # For CV later

# 3. Apply SMOTE to training data
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# 4. Train the model
rf_smote = RandomForestClassifier(
    n_estimators=100,
    max_depth=None,
    class_weight='balanced',
    random_state=42
)
rf_smote.fit(X_train_smote, y_train_smote)

# 5. Predictions
y_pred = rf_smote.predict(X_test)
y_proba = rf_smote.predict_proba(X_test)[:, 1]

# 6. Metrics
print("📊 Classification Report (Random Forest + SMOTE):\n")
print(classification_report(y_test, y_pred, target_names=["Dropout", "Graduate"]))

print("📌 Performance on Test Set:")
print(f"✅ Accuracy:   {accuracy_score(y_test, y_pred):.4f}")
print(f"✅ Precision:  {precision_score(y_test, y_pred):.4f}")
print(f"✅ Recall:     {recall_score(y_test, y_pred):.4f}")
print(f"✅ F1 Score:   {f1_score(y_test, y_pred):.4f}")
print(f"✅ ROC-AUC:    {roc_auc_score(y_test, y_proba):.4f}")
print(f"✅ MSE:        {mean_squared_error(y_test, y_proba):.4f}")
print(f"✅ Log Loss:   {log_loss(y_test, y_proba):.4f}")

# 7. Confusion Matrix
ConfusionMatrixDisplay.from_estimator(
    rf_smote, X_test, y_test, display_labels=["Dropout", "Graduate"], cmap='Blues'
)
plt.title("🔍 Confusion Matrix: RF + SMOTE")
plt.grid(False)
plt.show()

# 8. ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_proba)
roc_auc = roc_auc_score(y_test, y_proba)

plt.figure()
RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name="RF + SMOTE").plot()
plt.title("📈 ROC Curve: Random Forest + SMOTE")
plt.grid(True)
plt.tight_layout()
plt.show()

# 9. Cross-Validation (with imputed X)
f1_cv = cross_val_score(rf_smote, X_imputed, y, scoring='f1', cv=5)
roc_auc_cv = cross_val_score(rf_smote, X_imputed, y, scoring='roc_auc', cv=5)

print("🔁 Cross-Validation Results (5-Fold):")
print(f"📌 Mean F1 Score:   {f1_cv.mean():.4f} ± {f1_cv.std():.4f}")
print(f"📌 Mean ROC-AUC:    {roc_auc_cv.mean():.4f} ± {roc_auc_cv.std():.4f}")


## 5.2 CatBoost + SMOTE

In [ ]:
# =========================================
# 🐱 CatBoost with SMOTE — Full Metrics
# =========================================

from catboost import CatBoostClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, ConfusionMatrixDisplay,
    roc_auc_score, roc_curve, RocCurveDisplay,
    f1_score, accuracy_score, precision_score, recall_score,
    log_loss, mean_squared_error
)
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import pandas as pd

# 1. Split dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# 2. Impute missing values
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# 3. SMOTE oversampling
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X_train_imputed, y_train)

# 4. Train CatBoost model
model_cb_smote = CatBoostClassifier(verbose=0, random_seed=42, eval_metric='F1')
model_cb_smote.fit(X_smote, y_smote)

# 5. Predictions
y_pred = model_cb_smote.predict(X_test_imputed)
y_proba = model_cb_smote.predict_proba(X_test_imputed)[:, 1]

# 6. Evaluation Metrics
print("📊 Classification Report: CatBoost (SMOTE)\n")
print(classification_report(y_test, y_pred))

print("\n📌 Test Metrics:")
print(f"✅ F1 Score:   {f1_score(y_test, y_pred):.4f}")
print(f"✅ Accuracy:   {accuracy_score(y_test, y_pred):.4f}")
print(f"✅ Precision:  {precision_score(y_test, y_pred):.4f}")
print(f"✅ Recall:     {recall_score(y_test, y_pred):.4f}")
print(f"✅ ROC-AUC:    {roc_auc_score(y_test, y_proba):.4f}")
print(f"✅ MSE:        {mean_squared_error(y_test, y_proba):.4f}")
print(f"✅ Log Loss:   {log_loss(y_test, y_proba):.4f}")

# 7. Confusion Matrix
ConfusionMatrixDisplay.from_predictions(
    y_test, y_pred, display_labels=["Dropout", "Graduate"], cmap='Blues'
)
plt.title("🔍 Confusion Matrix: CatBoost (SMOTE)")
plt.grid(False)
plt.tight_layout()
plt.show()

# 8. ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_proba)
roc_auc = roc_auc_score(y_test, y_proba)

RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name="CatBoost (SMOTE)").plot()
plt.title("📈 ROC Curve: CatBoost (SMOTE)")
plt.grid(True)
plt.tight_layout()
plt.show()


## 5.3 ANN + SMOTE

In [ ]:
# =====================================
# 🤖 ANN + SMOTE (5-Fold CV) — Final Version with Imputation + SHAP
# =====================================

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    f1_score, accuracy_score, roc_auc_score, roc_curve, auc,
    precision_score, recall_score, log_loss, mean_squared_error, confusion_matrix
)
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import shap
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Setup
f1_scores, acc_scores, auc_scores = [], [], []
prec_scores, rec_scores, logloss_scores, mse_scores = [], [], [], []
conf_matrices = []

n_splits = 5
random_seed = 42
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_seed)

X_array = X.to_numpy()
y_array = y.to_numpy()

tprs, aucs = [], []
mean_fpr = np.linspace(0, 1, 100)
plt.figure(figsize=(8, 6))

for fold, (train_idx, test_idx) in enumerate(skf.split(X_array, y_array), 1):
    print(f"\n🔁 Fold {fold}/{n_splits}")

    # Split
    X_train, X_test = X_array[train_idx], X_array[test_idx]
    y_train, y_test = y_array[train_idx], y_array[test_idx]

    # Standardize
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Impute missing values (mean strategy)
    imputer = SimpleImputer(strategy='mean')
    X_train_scaled = imputer.fit_transform(X_train_scaled)
    X_test_scaled = imputer.transform(X_test_scaled)

    # SMOTE oversampling
    smote = SMOTE(random_state=random_seed)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

    # ANN model
    model = Sequential([
        Dense(16, activation='relu', input_shape=(X_train_resampled.shape[1],)),
        Dropout(0.2),
        Dense(8, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model.fit(
        X_train_resampled, y_train_resampled,
        validation_split=0.2,
        epochs=50,
        batch_size=32,
        callbacks=[early_stop],
        verbose=0
    )

    # Predictions
    y_pred_probs = model.predict(X_test_scaled).ravel()
    y_pred = (y_pred_probs >= 0.5).astype(int)

    # Metrics
    acc_scores.append(accuracy_score(y_test, y_pred))
    f1_scores.append(f1_score(y_test, y_pred))
    prec_scores.append(precision_score(y_test, y_pred))
    rec_scores.append(recall_score(y_test, y_pred))
    auc_val = roc_auc_score(y_test, y_pred_probs)
    auc_scores.append(auc_val)
    logloss_scores.append(log_loss(y_test, y_pred_probs))
    mse_scores.append(mean_squared_error(y_test, y_pred_probs))
    conf_matrices.append(confusion_matrix(y_test, y_pred))

    # ROC
    fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
    tprs.append(np.interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    aucs.append(auc_val)
    plt.plot(fpr, tpr, alpha=0.3, label=f'Fold {fold} ROC (AUC = {auc_val:.2f})')

# 🎯 Final ROC Curve
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
mean_tpr = np.mean(tprs, axis=0)
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
plt.plot(mean_fpr, mean_tpr, color='b', label=f'Mean ROC (AUC = {mean_auc:.2f} ± {std_auc:.2f})', lw=2)
plt.title('📈 ROC Curve - ANN + SMOTE (5-Fold CV)')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.grid(True)
plt.tight_layout()
plt.show()

# 🔁 Averaged Confusion Matrix
avg_cm = np.mean(conf_matrices, axis=0).astype(int)
plt.figure(figsize=(6, 5))
sns.heatmap(avg_cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Dropout', 'Graduate'],
            yticklabels=['Dropout', 'Graduate'])
plt.title("🔁 Averaged Confusion Matrix (ANN + SMOTE)")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.tight_layout()
plt.show()

# 🔍 SHAP with KernelExplainer (CPU-friendly)
X_sample_train = X_train_resampled[:100]
X_sample_test = X_test_scaled[:100]
predict_fn = lambda x: model.predict(x).ravel()

explainer = shap.KernelExplainer(predict_fn, X_sample_train)
shap_values = explainer.shap_values(X_sample_test)
shap.summary_plot(shap_values, features=X_sample_test, feature_names=X.columns)

# 📊 Final Metrics Summary
print("\n📊 ANN + SMOTE Summary:")
print(f"✅ Accuracy:   {np.mean(acc_scores):.4f} ± {np.std(acc_scores):.4f}")
print(f"✅ Precision:  {np.mean(prec_scores):.4f} ± {np.std(prec_scores):.4f}")
print(f"✅ Recall:     {np.mean(rec_scores):.4f} ± {np.std(rec_scores):.4f}")
print(f"✅ F1 Score:   {np.mean(f1_scores):.4f} ± {np.std(f1_scores):.4f}")
print(f"✅ ROC-AUC:    {np.mean(auc_scores):.4f} ± {np.std(auc_scores):.4f}")
print(f"✅ MSE:        {np.mean(mse_scores):.4f} ± {np.std(mse_scores):.4f}")
print(f"✅ Log Loss:   {np.mean(logloss_scores):.4f} ± {np.std(logloss_scores):.4f}")


# 6. ADASYN Oversampling Comparisons

## 6.1 Random Forrest + ADASYN

In [ ]:
# =========================================
# 🌲 Random Forest with ADASYN — Full Metrics (Fixed)
# =========================================

from imblearn.over_sampling import ADASYN
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report, ConfusionMatrixDisplay,
    roc_auc_score, roc_curve, RocCurveDisplay,
    accuracy_score, f1_score, precision_score, recall_score,
    log_loss, mean_squared_error
)
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt

# 1. Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# 2. Impute missing values (required for ADASYN)
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# 3. ADASYN oversampling
adasyn = ADASYN(random_state=42)
X_train_adasyn, y_train_adasyn = adasyn.fit_resample(X_train_imputed, y_train)

# 4. Train model
rf_adasyn = RandomForestClassifier(random_state=42)
rf_adasyn.fit(X_train_adasyn, y_train_adasyn)

# 5. Predictions
y_pred = rf_adasyn.predict(X_test_imputed)
y_proba = rf_adasyn.predict_proba(X_test_imputed)[:, 1]

# 6. Evaluation Metrics
print("📊 Classification Report: Random Forest (ADASYN)\n")
print(classification_report(y_test, y_pred))

print("\n📌 Test Metrics:")
print(f"✅ F1 Score:   {f1_score(y_test, y_pred):.4f}")
print(f"✅ Accuracy:   {accuracy_score(y_test, y_pred):.4f}")
print(f"✅ Precision:  {precision_score(y_test, y_pred):.4f}")
print(f"✅ Recall:     {recall_score(y_test, y_pred):.4f}")
print(f"✅ ROC-AUC:    {roc_auc_score(y_test, y_proba):.4f}")
print(f"✅ MSE:        {mean_squared_error(y_test, y_proba):.4f}")
print(f"✅ Log Loss:   {log_loss(y_test, y_proba):.4f}")

# 7. Confusion Matrix
ConfusionMatrixDisplay.from_predictions(
    y_test, y_pred, display_labels=["Dropout", "Graduate"], cmap='Blues'
)
plt.title("🔍 Confusion Matrix: Random Forest (ADASYN)")
plt.grid(False)
plt.tight_layout()
plt.show()

# 8. ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_proba)
roc_auc = roc_auc_score(y_test, y_proba)

RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name="RF (ADASYN)").plot()
plt.title("📈 ROC Curve: Random Forest (ADASYN)")
plt.grid(True)
plt.tight_layout()
plt.show()


## 6.2 CatBoost + ADASYN

In [ ]:
# =========================================
# 🐱 CatBoost with ADASYN — Full Metrics + Extended Error Analysis
# =========================================

from imblearn.over_sampling import ADASYN
from catboost import CatBoostClassifier
from sklearn.metrics import (
    classification_report, ConfusionMatrixDisplay,
    roc_auc_score, roc_curve, RocCurveDisplay,
    accuracy_score, f1_score, precision_score, recall_score,
    log_loss, mean_squared_error
)
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from scipy.stats import ttest_ind
from matplotlib import pyplot as plt
import seaborn as sns
import shap
import numpy as np
import pandas as pd

# 1. Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# 2. Impute missing values
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# 2.1 Handle possible column mismatch
X_columns = X.columns
if X_test_imputed.shape[1] != len(X_columns):
    X_columns = X_columns[:X_test_imputed.shape[1]]
X_test_df = pd.DataFrame(X_test_imputed, columns=X_columns)

# 3. ADASYN Oversampling
adasyn = ADASYN(random_state=42)
X_train_adasyn, y_train_adasyn = adasyn.fit_resample(X_train_imputed, y_train)

# 4. Train CatBoost
cat_model = CatBoostClassifier(verbose=0, random_state=42)
cat_model.fit(X_train_adasyn, y_train_adasyn)

# 5. Predictions
y_pred = cat_model.predict(X_test_imputed)
y_proba = cat_model.predict_proba(X_test_imputed)[:, 1]

# 6. Evaluation Metrics
print("📊 Classification Report: CatBoost (ADASYN)\n")
print(classification_report(y_test, y_pred))

print("\n📌 Test Metrics:")
print(f"✅ F1 Score:   {f1_score(y_test, y_pred):.4f}")
print(f"✅ Accuracy:   {accuracy_score(y_test, y_pred):.4f}")
print(f"✅ Precision:  {precision_score(y_test, y_pred):.4f}")
print(f"✅ Recall:     {recall_score(y_test, y_pred):.4f}")
print(f"✅ ROC-AUC:    {roc_auc_score(y_test, y_proba):.4f}")
print(f"✅ MSE:        {mean_squared_error(y_test, y_proba):.4f}")
print(f"✅ Log Loss:   {log_loss(y_test, y_proba):.4f}")

# 7. Confusion Matrix
ConfusionMatrixDisplay.from_predictions(
    y_test, y_pred, display_labels=["Dropout", "Graduate"], cmap='Blues'
)
plt.title("🔍 Confusion Matrix: CatBoost (ADASYN)")
plt.grid(False)
plt.tight_layout()
plt.show()

# 8. ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_proba)
roc_auc = roc_auc_score(y_test, y_proba)

RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name="CatBoost (ADASYN)").plot()
plt.title("📈 ROC Curve: CatBoost (ADASYN)")
plt.grid(True)
plt.tight_layout()
plt.show()

# 9. SHAP Analysis
print("\n🔎 SHAP Summary Plot (Class: Graduate):")
explainer = shap.TreeExplainer(cat_model)
shap_values = explainer.shap_values(X_test_df)

# Plot SHAP summary and store for reuse
if isinstance(shap_values, list):
    shap.summary_plot(shap_values[1], X_test_df, plot_type="bar")
    shap_array = shap_values[1]
else:
    shap.summary_plot(shap_values, X_test_df, plot_type="bar")
    shap_array = shap_values

# Top feature for follow-up
top_feature_index = np.argmax(np.abs(np.mean(shap_array, axis=0)))
top_feature = X_columns[top_feature_index]

# 10. SHAP Dependence Plot
print(f"\n📈 SHAP Dependence Plot for: {top_feature}")
shap.dependence_plot(top_feature, shap_array, X_test_df)

# 11. Align indices
y_test_series = pd.Series(y_test, index=X_test_df.index)
y_pred_series = pd.Series(y_pred, index=X_test_df.index)

# 12. TP and FN Identification
TP_idx = (y_test_series == 1) & (y_pred_series == 1)
FN_idx = (y_test_series == 1) & (y_pred_series == 0)

tp_values = X_test_df.loc[TP_idx, top_feature]
fn_values = X_test_df.loc[FN_idx, top_feature]

# 13. Cohen’s d
mean_diff = tp_values.mean() - fn_values.mean()
pooled_std = np.sqrt((tp_values.std()**2 + fn_values.std()**2) / 2)
cohen_d = mean_diff / pooled_std
print(f"\n📐 Cohen’s d for '{top_feature}' between TP and FN: {cohen_d:.2f}")

# 14. T-test
t_stat, p_val = ttest_ind(tp_values, fn_values, equal_var=False)
print(f"📊 T-test p-value for '{top_feature}': {p_val:.4f}")

# 15. Plot distribution: TP vs FN for top feature
plt.figure(figsize=(8, 5))
sns.kdeplot(tp_values, label="True Positives", fill=True)
sns.kdeplot(fn_values, label="False Negatives", fill=True)
plt.title(f"🔍 Distribution of '{top_feature}' for TP vs FN")
plt.xlabel(top_feature)
plt.legend()
plt.tight_layout()
plt.show()

# 16. Error Breakdown by Gender (or other categorical)
if "Gender" in X_test_df.columns:
    gender_df = X_test_df.copy()
    gender_df["TrueLabel"] = y_test_series
    gender_df["Prediction"] = y_pred_series
    gender_df["Correct"] = gender_df["TrueLabel"] == gender_df["Prediction"]
    
    gender_accuracy = gender_df.groupby("Gender")["Correct"].mean()
    print("\n📊 Accuracy by Gender:")
    print(gender_accuracy)

    gender_counts = gender_df.groupby(["Gender", "Correct"]).size().unstack()
    gender_counts.plot(kind="bar", stacked=True)
    plt.title("🔍 Prediction Correctness by Gender")
    plt.ylabel("Count")
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ Column 'Gender' not found for subgroup analysis.")



## 6.3 ANN + ADASYN


In [ ]:
# =====================================
# 🤖 ANN with ADASYN — Full Evaluation + SHAP
# =====================================

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    f1_score, accuracy_score, roc_auc_score, roc_curve, auc,
    precision_score, recall_score, log_loss, mean_squared_error, confusion_matrix
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from imblearn.over_sampling import ADASYN

# Assumes X and y are already defined as pandas DataFrames/Series
X_array = X.to_numpy()
y_array = y.to_numpy()

f1_scores, acc_scores, auc_scores = [], [], []
prec_scores, rec_scores, logloss_scores, mse_scores = [], [], [], []
conf_matrices = []

n_splits = 5
random_seed = 42
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_seed)

tprs, aucs = [], []
mean_fpr = np.linspace(0, 1, 100)
plt.figure(figsize=(8, 6))

for fold, (train_idx, test_idx) in enumerate(skf.split(X_array, y_array), 1):
    print(f"\n🔁 Fold {fold}/{n_splits}")
    X_train, X_test = X_array[train_idx], X_array[test_idx]
    y_train, y_test = y_array[train_idx], y_array[test_idx]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # ✅ Impute missing values before using ADASYN
    imputer = SimpleImputer(strategy='mean')
    X_train_scaled = imputer.fit_transform(X_train_scaled)
    X_test_scaled = imputer.transform(X_test_scaled)

    adasyn = ADASYN(random_state=random_seed)
    X_train_resampled, y_train_resampled = adasyn.fit_resample(X_train_scaled, y_train)

    model = Sequential([
        Dense(16, activation='relu', input_shape=(X_train_resampled.shape[1],)),
        Dropout(0.2),
        Dense(8, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train_resampled, y_train_resampled, validation_split=0.2, epochs=50, batch_size=32,
              callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)],
              verbose=0)

    y_pred_probs = model.predict(X_test_scaled).ravel()
    y_pred = (y_pred_probs >= 0.5).astype(int)

    acc_scores.append(accuracy_score(y_test, y_pred))
    f1_scores.append(f1_score(y_test, y_pred))
    prec_scores.append(precision_score(y_test, y_pred))
    rec_scores.append(recall_score(y_test, y_pred))
    auc_val = roc_auc_score(y_test, y_pred_probs)
    auc_scores.append(auc_val)
    logloss_scores.append(log_loss(y_test, y_pred_probs))
    mse_scores.append(mean_squared_error(y_test, y_pred_probs))
    conf_matrices.append(confusion_matrix(y_test, y_pred))

    fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
    tprs.append(np.interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    aucs.append(auc_val)
    plt.plot(fpr, tpr, alpha=0.3, label=f'Fold {fold} ROC (AUC = {auc_val:.2f})')

# 📈 ROC Curve
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
mean_tpr = np.mean(tprs, axis=0)
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
plt.plot(mean_fpr, mean_tpr, color='b', label=f'Mean ROC (AUC = {mean_auc:.2f} ± {std_auc:.2f})', lw=2)
plt.title('📈 ROC Curve - ANN + ADASYN (5-Fold CV)')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.grid(True)
plt.tight_layout()
plt.show()

# 📊 Averaged Confusion Matrix
avg_cm = np.mean(conf_matrices, axis=0).astype(int)
plt.figure(figsize=(6, 5))
sns.heatmap(avg_cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Dropout', 'Graduate'],
            yticklabels=['Dropout', 'Graduate'])
plt.title("🔁 Averaged Confusion Matrix (ANN + ADASYN)")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.tight_layout()
plt.show()

# ✅ SHAP (KernelExplainer — CPU compatible)
X_train_sample = X_train_resampled[:100]
X_test_sample = X_test_scaled[:100]
predict_fn = lambda x: model.predict(x).ravel()

explainer = shap.KernelExplainer(predict_fn, X_train_sample)
shap_values = explainer.shap_values(X_test_sample)
shap.summary_plot(shap_values, features=X_test_sample, feature_names=X.columns)

# 📊 Summary Metrics
print("\n📊 ANN + ADASYN Summary:")
print(f"✅ Accuracy:   {np.mean(acc_scores):.4f} ± {np.std(acc_scores):.4f}")
print(f"✅ Precision:  {np.mean(prec_scores):.4f} ± {np.std(prec_scores):.4f}")
print(f"✅ Recall:     {np.mean(rec_scores):.4f} ± {np.std(rec_scores):.4f}")
print(f"✅ F1 Score:   {np.mean(f1_scores):.4f} ± {np.std(f1_scores):.4f}")
print(f"✅ ROC-AUC:    {np.mean(auc_scores):.4f} ± {np.std(auc_scores):.4f}")
print(f"✅ MSE:        {np.mean(mse_scores):.4f} ± {np.std(mse_scores):.4f}")
print(f"✅ Log Loss:   {np.mean(logloss_scores):.4f} ± {np.std(logloss_scores):.4f}")
